In [32]:
#import packages
import pickle
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
#read in .josn files 
with open('data/yelp-dataset/yelp_academic_dataset_business.json') as fp: #请保持data/yelp-dataset/xxxx 的路径，方便交流
    lines = fp.readlines()

In [57]:
# Create dataframe
list_ = []
for i_line in lines:
    list_.append(json.loads(i_line))
df_business = pd.DataFrame(list_)
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 15 columns):
address         188593 non-null object
attributes      162807 non-null object
business_id     188593 non-null object
categories      188052 non-null object
city            188593 non-null object
hours           143791 non-null object
is_open         188593 non-null int64
latitude        188587 non-null float64
longitude       188587 non-null float64
name            188593 non-null object
neighborhood    188593 non-null object
postal_code     188593 non-null object
review_count    188593 non-null int64
stars           188593 non-null float64
state           188593 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 21.6+ MB


In [58]:
#filter cities and drop NaN
df_business = df_business[(df_business['city']=='Toronto')
                          | (df_business['city']=='Las Vegas')
                          | (df_business['city']=='Cleveland')
                          | (df_business['city']=='Montréal')]
print('with NaN', df_business.shape)
print('withou NaN', df_business.dropna().shape)
df_business.dropna(inplace=True)
df_business.head()

with NaN (56649, 15)
withou NaN (38117, 15)


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
11,595 Markham Street,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",tZnSodhPwNr4bzrwJ1CSbw,"Cajun/Creole, Southern, Restaurants",Toronto,"{'Tuesday': '17:0-1:0', 'Wednesday': '17:0-1:0...",0,43.664125,-79.411886,Southern Accent Restaurant,Palmerston,M6G 2L7,146,4.0,ON
16,438 Place Jacques Cartier,"{'Alcohol': 'full_bar', 'BikeParking': 'True',...",r89Re4FNgVWHgBfjCVZyVw,"Nightlife, Bars, American (Traditional), Tapas...",Montréal,"{'Monday': '11:30-23:0', 'Tuesday': '11:30-23:...",0,45.507699,-73.553407,Harlow,Ville-Marie,H2Y 3B3,12,3.5,QC
19,"3940 Martin Luther King Blvd, Ste 101","{'BikeParking': 'True', 'BusinessAcceptsCredit...",vJIuDBdu01vCA8y1fwR1OQ,"American (Traditional), Food, Bakeries, Restau...",Las Vegas,"{'Tuesday': '11:0-18:0', 'Wednesday': '11:0-18...",0,36.192284,-115.159272,CakesbyToi,,89106,3,1.5,NV
23,746 Street Clair Avenue W,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",5J3b7j3Fzo9ISjChmoUoUA,"Food, Bakeries, Coffee & Tea",Toronto,"{'Monday': '7:30-19:0', 'Tuesday': '7:30-19:0'...",1,43.681328,-79.427884,Mabel's Bakery,Wychwood,M6C 1B5,23,4.0,ON


In [62]:
# output filtered dataframe to binary files as pickles, so that one may use it to filter out dataset
with open('data/yelp-dataset/df_business.pickle', 'wb') as fp:
    pickle.dump(df_business['business_id'],fp)

In [60]:
df_business.groupby('city').count().sort_values(by='stars', ascending=False)

,address,attributes,business_id,categories,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
city,,,,,,,,,,,,,,
Las Vegas,19613,19613,19613,19613,19613,19613,19613,19613,19613,19613,19613,19613,19613,19613
Toronto,12156,12156,12156,12156,12156,12156,12156,12156,12156,12156,12156,12156,12156,12156
Montréal,4147,4147,4147,4147,4147,4147,4147,4147,4147,4147,4147,4147,4147,4147
Cleveland,2201,2201,2201,2201,2201,2201,2201,2201,2201,2201,2201,2201,2201,2201


In [56]:
type(df_business.hours.iloc[0])

dict

In [50]:
df_business.hours.iloc[0]

{'Monday': '10:0-22:0',
 'Tuesday': '10:0-22:0',
 'Wednesday': '10:0-22:0',
 'Thursday': '10:0-22:0',
 'Friday': '10:0-22:0',
 'Saturday': '10:0-22:0',
 'Sunday': '10:0-22:0'}

#data cleanning...